# EcoCyc E. coli Biochemical Synthesis

The following is the synthesis script for the **WHOLE-CELL BIOCHEMICAL RECONSTRUCTION MODEL** project that was conducted by Carson Billingsley from Fall 2023 - Spring 2024. This was part of the M.Eng Bioengineering Capstone #98, with the overarching title: "Software Development for Synthetic Biology".

**Project Objective:** The overarching goal of this project was to program a whole-cell biochemical pathway simulation library that can model and analyze the metabolism of commonly used model organisms in synthetic biology. I used Python to build this biochemical network reconstruction and analysis tool, and Escherichia coli was selected as the proof-of-concept model organism. The software module was built based on a flat file data release of EcoCyc, a comprehensive database made by BioCyc that comprises the known metabolic pathways, enzymes, metabolites, and reactions of E. coli. My program employs an in-house synthesis algorithm that was originally written at UC Berkeley that systematically builds the metabolic pathways based on a set of predetermined native metabolites. This whole-cell reconstruction model has a variety of applications, such as 1) being a critical piece to make future SAR-RO enzyme models more biochemically accurate and effective at determining biosynthetically reachable chemicals in an organism or 2) as the foundation for a transformation validator software that predicts the biological processes initiated upon the entry of DNA into the cell.

Please email Carson with any specific questions: billingsley.carson@gmail.com

# Installing Dependancies

In [1]:
# Install and import libraries
%%capture
!pip install rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import rdChemReactions
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import MolStandardize, inchi
from rdkit.Chem.MolStandardize import rdMolStandardize, tautomer
from rdkit import RDLogger

from concurrent.futures import ThreadPoolExecutor, TimeoutError

import numpy as np

from dataclasses import dataclass
from typing import List
from typing import Tuple

from IPython.display import display
from IPython.display import Image

# Downloading and Reading Drive Files with GDown

Using GDown to upload and read the two flat files sourced from BioCyc (**compounds.dat** and r**eactions.dat**) These two documents came from the EcoCyc flat file release that came to Dr. Anderson in an email on Oct. 10, 2023.

The custom lists of minimal metabolites and universal metabolites, to occupy Shell 0, are also uploaded using this method.

In [2]:
# !pip install -q gdown --upgrade

# import gdown

# def download_file(file_id, output_filename):
#     """
#     Download a file from Google Drive using its ID.

#     :param file_id: str, Google Drive file ID.
#     :param output_filename: str, Name to save the downloaded file.
#     """
#     # Construct the download URL
#     url = f'https://drive.google.com/uc?id={file_id}'

#     # Download the file
#     gdown.download(url, output_filename, quiet=False)

# def preview_file(output_filename, num_lines=5):
#     """
#     Print the first n lines of a file.

#     :param output_filename: str, Name of the file to preview.
#     :param num_lines: int, Number of lines to preview. Default is 5.
#     """
#     try:
#         with open(output_filename, 'r') as file:
#             print(f"Previewing first {num_lines} lines of {output_filename}:")
#             for _ in range(num_lines):
#                 print(next(file).strip())
#             print("\n")
#     except Exception as e:
#         print(f"An error occurred while previewing the file: {str(e)}")
#     except StopIteration:
#         print(f"{output_filename} may have less than {num_lines} lines to preview.\n")

# # File IDs and names
# files_info = {
#     "compounds.dat": '1mB-xIewC1UY3mLzVi0ovRV6bQPS7RzYg',
#     "reactions.dat": '1SjVODv98B3KJxV0hFrNAPN0B-3nTfr1b',
#     "minimal_metabolites_02-24.txt": '1PQazv8Q0cjn_ug76mIICyxNy7BxNJwds',
#     "universal_metabolites_03-24.txt": '1jlLnO7oAmz4dqaynmPBegCSAPXemPB-R'
# }

# # Download and preview the files
# for filename, file_id in files_info.items():
#     download_file(file_id, filename)

# # Preview only the chemicals file
# preview_file("compounds.dat")

Sometimes the GDown method is finnicky, so WGet is another perfectly acceptable option.

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1mB-xIewC1UY3mLzVi0ovRV6bQPS7RzYg' -O compounds.dat
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1SjVODv98B3KJxV0hFrNAPN0B-3nTfr1b' -O reactions.dat
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1PQazv8Q0cjn_ug76mIICyxNy7BxNJwds' -O minimal_metabolites_02-24.txt
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1jlLnO7oAmz4dqaynmPBegCSAPXemPB-R' -O universal_metabolites_03-24.txt

#Defining EcoCyc Chemical and EcoCyc Reaction Classes

In [4]:
@dataclass(frozen=True)
class EcoCyc_Chemical:
    UNIQUE_ID: str = ""
    COMMON_NAME: str = ""
    SMILES: str = ""
    INCHI: str = ""
    NON_STANDARD_INCHI: str = ""

# Note: Since dataclasses automatically generate __repr__, you don't need to define it unless you want a custom representation.


@dataclass(frozen=True)
class EcoCyc_Reaction:
    UNIQUE_ID: str = ""
    LEFT: List[str] = None
    RIGHT: List[str] = None
    EC_NUMBER: str = ""
    REACTION_DIRECTION: str = ""

    def __post_init__(self): #used to ensure that SUBSTRATES and PRODUCTS are initialized as empty lists if they are not provided during object creation (safer for list operations)
        if self.LEFT is None:
            self.LEFT = []
        if self.RIGHT is None:
            self.RIGHT = []


# Parsing the EcoCyc files into EcoCyc objects

###EcoCyc_Chemicals from Compounds.dat

Using a parser of **compounds.dat** to construct the EcoCyc_Chemical ontology. The EcoCyc_Chemical class and cooresponding objects is meant to be an exact replica of what is found in the raw data. These objects are stored in a dictionary called **ecocyc_chemicals_dict**



In [5]:
def parse_chemicals_to_dict(file_contents):
    ecocyc_chemicals_dict = {}

    ecocyc_chemical_data = {
        'UNIQUE_ID': '',
        'COMMON_NAME': '',
        'SMILES': '',
        'INCHI': '',
        'NON_STANDARD_INCHI': '',
    }

    for line in file_contents:
        if '- ' in line:
            key, value = line.split('- ', 1)  # 1 is used as a max split to limit the number of splits
            key = key.strip().replace('-', '_').upper()  # Converting to dataclass format
            value = value.strip()

            if key in ecocyc_chemical_data:
                ecocyc_chemical_data[key] = value #Assigns values to the corresponding keys in the ecocyc_chemical_data dictionary
        elif line.strip() == '//':  # End of chemical data entry in the flat file
            chem_obj = EcoCyc_Chemical(**ecocyc_chemical_data) #Creates an instance of EcoCyc_Chemical using the data stored in ecocyc_chemical_data.
            #The ** operator is used to unpack the dictionary into keyword arguments
            ecocyc_chemicals_dict[ecocyc_chemical_data['UNIQUE_ID']] = chem_obj #The new EcoCyc_Chemical object is stored in ecocyc_chemicals_dict dictionary; UNIQUE_ID is the key.
            ecocyc_chemical_data = {k: '' for k in ecocyc_chemical_data}  # Reset for the next entry

    return ecocyc_chemicals_dict #In the dictionary: Key=UNIQUE_ID, Value= the entire EcoCyc_Chemical object

# Code to read file and create dictionary
with open("compounds.dat", "r", encoding="ISO-8859-1") as file:
    file_contents = file.readlines()
ecocyc_chemicals_dict = parse_chemicals_to_dict(file_contents)

# Test by printing the first 5 chemicals
for test_key in list(ecocyc_chemicals_dict.keys())[:5]: #Specifically creating list of the keys in ecocyc_chemicals_dict dictionary.
    print(ecocyc_chemicals_dict[test_key])


EcoCyc_Chemical(UNIQUE_ID='Ureidocarboxylates', COMMON_NAME='an ureidocarboxylate', SMILES='', INCHI='', NON_STANDARD_INCHI='')
EcoCyc_Chemical(UNIQUE_ID='Monogalactosyldiacylglycerols-38-3', COMMON_NAME='monogalactosyldiacylglycerol-38:3', SMILES='', INCHI='', NON_STANDARD_INCHI='')
EcoCyc_Chemical(UNIQUE_ID='6-H-5-I-2-methylhexanoates', COMMON_NAME='a 6-hydroxy-5-isopropenyl-2-methylhexanoate', SMILES='C=C(C)C(CCC(C)C(=O)[O-])CO', INCHI='InChI=1S/C10H18O3/c1-7(2)9(6-11)5-4-8(3)10(12)13/h8-9,11H,1,4-6H2,2-3H3,(H,12,13)/p-1', NON_STANDARD_INCHI='InChI=1S/C10H18O3/c1-7(2)9(6-11)5-4-8(3)10(12)13/h8-9,11H,1,4-6H2,2-3H3,(H,12,13)')
EcoCyc_Chemical(UNIQUE_ID='Monogalactosyldiacylglycerols-36-4', COMMON_NAME='monogalactosyldiacylglycerol-36:4', SMILES='', INCHI='', NON_STANDARD_INCHI='')
EcoCyc_Chemical(UNIQUE_ID='Short-Chain-Carboxylates', COMMON_NAME='a short-chain carboxylate', SMILES='C([O-])(=O)[R]', INCHI='', NON_STANDARD_INCHI='InChI=1S/CH2O2/c2-1-3/h1H,(H,2,3)/p-1')


###EcoCyc_Reactions from Reactions.dat

Using a parser of **reactions.dat** to construct the EcoCyc_Reaction ontology. The EcoCyc_Reaction class and cooresponding objects is meant to be an exact replica of what is found in the raw data. These objects are stored in a dictionary called **ecocyc_reactions_dict**




In [6]:
def parse_reactions_to_dict(file_contents):
    ecocyc_reactions_dict = {}
    ecocyc_reaction_data = {
        'UNIQUE_ID': '',
        'LEFT': [],
        'RIGHT': [],
        'EC_NUMBER': '',
        'REACTION_DIRECTION': ''
    }

    for line in file_contents:
        if '- ' in line:
            key, value = line.split('- ', 1)
            key = key.strip().replace('-', '_').upper()
            value = value.strip()

            if key in ['UNIQUE_ID', 'EC_NUMBER', 'REACTION_DIRECTION']:
                ecocyc_reaction_data[key] = value
            elif key in ['LEFT', 'RIGHT']:
                ecocyc_reaction_data[key].append(value) #Must append because we are making a list
        elif line.strip() == '//':  # End of reaction data entry
            reaction_obj = EcoCyc_Reaction(**ecocyc_reaction_data)
            ecocyc_reactions_dict[ecocyc_reaction_data['UNIQUE_ID']] = reaction_obj
            ecocyc_reaction_data = {
                'UNIQUE_ID': '',
                'LEFT': [],
                'RIGHT': [],
                'EC_NUMBER': '',
                'REACTION_DIRECTION': ''
            } # Reset for the next entry

    return ecocyc_reactions_dict

# Code to read file and create dictionary
with open("reactions.dat", "r", encoding="ISO-8859-1") as file:
    file_contents = file.readlines()
ecocyc_reactions_dict = parse_reactions_to_dict(file_contents)

# Test by printing the first 5 reactions
for key in list(ecocyc_reactions_dict.keys())[:5]:
    print(ecocyc_reactions_dict[key])


EcoCyc_Reaction(UNIQUE_ID='RXN-21279', LEFT=['a-5-deoxyribose-5-phosphate-DNA'], RIGHT=['5-Phospho-terminated-DNAs', 'CPD-22978', 'PROTON'], EC_NUMBER='', REACTION_DIRECTION='PHYSIOL-LEFT-TO-RIGHT')
EcoCyc_Reaction(UNIQUE_ID='RXN-19954', LEFT=['2Cys-Peroxiredoxins-With-HydroxyCys'], RIGHT=['Cys2-Peroxiredoxin-Disulfide', 'WATER'], EC_NUMBER='', REACTION_DIRECTION='PHYSIOL-LEFT-TO-RIGHT')
EcoCyc_Reaction(UNIQUE_ID='RXN-17919', LEFT=['A-5-prime-PP-5-prime-DNA', '3-Hydroxy-Terminated-DNAs'], RIGHT=['DNA-N', 'AMP', 'PROTON'], EC_NUMBER='', REACTION_DIRECTION='PHYSIOL-LEFT-TO-RIGHT')
EcoCyc_Reaction(UNIQUE_ID='RXN0-4701', LEFT=['mRNA-Holder', 'WATER'], RIGHT=['ssRNA-with-3phosphate', 'ssRNA-with-5OH'], EC_NUMBER='', REACTION_DIRECTION='LEFT-TO-RIGHT')
EcoCyc_Reaction(UNIQUE_ID='RXN-20692', LEFT=['Cys2-Peroxiredoxin-Disulfide', 'Red-Prx-Disulfide-Reductases'], RIGHT=['Reduced-Cys2-Peroxiredoxins', 'Ox-Prx-Disulfide-Reductases'], EC_NUMBER='', REACTION_DIRECTION='PHYSIOL-LEFT-TO-RIGHT')


# Creating Chemical and Reaction Objects for Synthesis

I'm making this a step because in the final product, there will be multiple alteration steps of the EcoCyc ontology compared with what we want in the end Chemical and Reaction ojects that are used directly in the synthesis algorithm. This is what we do in the follow section. See the entire documentation for more details.

In [7]:
# @dataclass(frozen=True)
# class Chemical:
#     UNIQUE_ID: str = ""
#     COMMON_NAME: str = ""
#     SMILES: str = ""
#     INCHI: str = ""
#     NON_STANDARD_INCHI: str = ""

@dataclass(frozen=True)
class Chemical:
    UNIQUE_ID: str = ""
    COMMON_NAME: str = ""
    SMILES: str = ""
    INCHI: str = ""
    NON_STANDARD_INCHI: str = ""
    HAS_NS_INCHI: bool = False
    HAS_SMILES: bool = False
    HAS_SMIRKS: bool = False

@dataclass(frozen=True)
class Reaction:
    UNIQUE_ID: str
    SUBSTRATES: Tuple[Chemical, ...]  # Assuming Chemical objects are hashable
    PRODUCTS: Tuple[Chemical, ...]
    EC_NUMBER: str
    REACTION_DIRECTION: str



# Data Processing and Standardization

### RDKit Standardization for Data Filtering

This step populates the Chemical and Reaction objects by incorporating a filtering algorithm of molecular standardization. The 'std' function calls a Mol object off each substrate's inchi and product's inchi present in an EcoCyc_Reaction. Those reactions that have chemicals that cannot be turned into Mol objects, don't have inchis, or cannot be standardized are thrown out. Standardized inchis are then used to populate a new **dictionary** with Chemical objects and Inchi keys, "**Chemicals_by_Inchi**", and remaining reactions with standardized chemicals end up as Reaction objects in the **"reactions_list"** **list**. Note: Reaction objects have exact Chemical objects as a tuple listed under substrates and products.

In [8]:
def normalize_molecule(mol):
    """
    Normalize a molecule using RDKit's Normalizer.
    Create a normalizer object, then normalize the molecule

    Parameters:
    mol (rdkit.Chem.Mol): The molecule to normalize.

    Returns:
    rdkit.Chem.Mol: The normalized molecule.
    """
    normalizer = rdMolStandardize.Normalizer()
    normalized_mol = normalizer.normalize(mol)

    return normalized_mol

def remove_small_fragments(mol):
    """
    Remove small fragments from a molecule, keeping only the largest fragment.
    Create a LargestFragmentChooser object, then select the largest fragment

    Parameters:
    mol (rdkit.Chem.Mol): The molecule from which to remove small fragments.

    Returns:
    rdkit.Chem.Mol: The molecule with only the largest fragment remaining.
    """
    fragment_parent = MolStandardize.fragment.LargestFragmentChooser()
    largest_fragment= fragment_parent.choose(mol)
    return largest_fragment

def neutralize_charges(mol):
    """
    Neutralize charges in a molecule using RDKit's Uncharger.
    Create an Uncharger object, then apply the uncharger to neutralize the molecule

    Parameters:
    mol (rdkit.Chem.Mol): The molecule to neutralize.

    Returns:
    rdkit.Chem.Mol: The neutralized molecule.
    """
    uncharger = rdMolStandardize.Uncharger()
    neutralized_mol = uncharger.uncharge(mol)

    return neutralized_mol

def standardize_aromatization(mol):
    """
    Standardize the aromatization state of a molecule.
    Kekulize the molecule, if possible, to define explicit bonds
    If kekulization fails (e.g., for some aromatic systems), catch the exception
      and no action is needed. The molecule remains unchanged

    Parameters:
    mol (rdkit.Chem.Mol): The molecule to standardize.

    Returns:
    rdkit.Chem.Mol: The molecule with a standardized aromatization state.
    """

    try:
        Chem.Kekulize(mol, clearAromaticFlags=True)
    except:
        pass

    # Set aromaticity flags based on RDKit's model
    Chem.SetAromaticity(mol, Chem.AromaticityModel.AROMATICITY_MDL)

    return mol

def standardize_tautomerization(mol):
    """
    Standardize tautomerization of a molecule.
    Create a tautomer enumerator, then get the canonical tautomer

    Parameters:
    mol (rdkit.Chem.Mol): The molecule to standardize.

    Returns:
    rdkit.Chem.Mol: The standardized (canonical) tautomer.
    """
    enumerator = rdMolStandardize.TautomerEnumerator()
    canonical_tautomer = enumerator.Canonicalize(mol)

    return canonical_tautomer

def standardize_tautomerization_with_timeout(mol, timeout=10):
    """
    Attempt to standardize tautomerization with a specified timeout.
    Returns the standardized molecule or None if a timeout/error occurs.
    """
    try:
        with ThreadPoolExecutor(max_workers=1) as executor:
            future = executor.submit(standardize_tautomerization, mol)
            return future.result(timeout=timeout)  # Timeout specified in seconds
    except Exception as e:
        # Catch both TimeoutError and any other exceptions
        return None

def apply_smirks_transformation(mol, smirks):
    """
    Attempt to apply a SMIRKS transformation to a molecule.
    If the transformation is applicable, return the first product.
    If not, return the original molecule.
    """
    reaction_operator = rdChemReactions.ReactionFromSmarts(smirks)
    products = reaction_operator.RunReactants((mol,))

    if products:
        # Assuming the first product of the first set is the desired outcome
        first_product_set = products[0]
        if first_product_set:
            return first_product_set[0]

    # If no products or transformation not applicable, return original molecule
    return mol

In [9]:
# Initialize an RDKit logger
rd_logger = RDLogger.logger()
rd_logger.setLevel(RDLogger.CRITICAL)  # Set the logger level to CRITICAL to capture errors

#Code provided to standardize the inchi
error_count = 0  # Initialize an error counter

tautomerization_failures = []

def std(inchi, return_intermediate=False):
    global error_count, tautomerization_failures
    intermediate_inchi = None
    final_inchi = None

    try:

      # assert isinstance(inchi, str), "InChI must be a string"
      mol = Chem.MolFromInchi(str(inchi))

      # if Inchi is deemed invalid
      if mol is None:
          error_count += 1
          return (None, None) if return_intermediate else None


      #Add hydrogens
      #mol = Chem.AddHs(mol)

      #Normalize by conducting a series of normalization transformations
      mol = normalize_molecule(mol)

      # Remove small fragments, keeping the largest one
      mol = remove_small_fragments(mol)

      # Neutralize charges
      mol = neutralize_charges(mol)

      #Standardize aromatization state
      mol = standardize_aromatization(mol)

      #Conversion of cyclical pyranose and furanose sugars to open-chain form
      mol = Chem.AddHs(mol)
      smirks_pyranose = '[C:1]1-[C:2]-[C:3]-[C:4]-[C:5](-[O:6]-[H])-[O:7]-1>>[O:7](-[H])-[C:1]-[C:2]-[C:3]-[C:4]-[C:5]=[O:6]'
      smirks_furanose = '[C:1]1-[C:2]-[C:3]-[C:5](-[O:6]-[H])-[O:7]-1>>[O:7](-[H])-[C:1]-[C:2]-[C:3]-[C:5]=[O:6]'
      mol = apply_smirks_transformation(mol, smirks_pyranose)
      mol = apply_smirks_transformation(mol, smirks_furanose)

      # Convert back to InChI to capture the state right before tautomerization
      intermediate_inchi = Chem.MolToInchi(mol)

      # Standardize tautomerization with a timeout mechanism
      mol = standardize_tautomerization_with_timeout(mol)
      if mol is None:
          # Log the InChI of the compound that failed tautomerization
          tautomerization_failures.append((inchi, intermediate_inchi))
          error_count += 1
          return (None, intermediate_inchi) if return_intermediate else None

      # Convert the final molecule back to InChI
      final_inchi = Chem.MolToInchi(mol)
      return (final_inchi, intermediate_inchi) if return_intermediate else final_inchi


    except Exception as e:
      error_count += 1
      rd_logger.error(f"Error processing InChI: {inchi} - {e}")
      return (None, None) if return_intermediate else None



###Two Options for Data Filtering

There are two routes for going about this code:

The first route is labeled CONCRETE ONLY, and it is currently listed last and currently commented out. Here is the psedo code: It iterates through all of the EcoCyc reactions. First, if the reaction doesn't have any substrates OR if the reaction does not have an associated an Inchi within the EcoCyc_Chemicals_dictionary for that substrate,  reaction_valid = False and that reaction is thrown out. However, if there is an Inchi, we attempt to employ my standardizer code. If we do not get it to standardize to a valid Inchi,  reaction_valid = False and that reaction is thrown out. But if it is successful, as long as a standardized version is not already there, we log the Inchi into chemicals_by_inchi as the key and Chemical object as the value (with the new standardized inchi. This process is repeated by iterating through products
If reaction remains valid, we stick these as substrates and products into the new Reaction object and append it to reactions_list. Both of these data structures (chemicals_by_inchi and reactions_list) are employed directly in the synthesis algorithm.


---------------------------------


The other version, listed second, is labeled RETAINS CLASS COMPOUNDS. This version of the code sequentially  processes each component of EcoCyc reactions, first attempting to standardize chemicals using their standard InChIs. If a chemical doesn't have a standard InChI, it progresses to check for a non-standard InChI, then a SMILES string. If the SMILES string is present but doesn't convert to a molecule, the code attempts to interpret it as SMARTS. Each successfully processed chemical is converted from an EcoCyc_Chemical to a Chemical object with the proper Boolean flag flipped to true. If all substrates and products of a reaction are successfully processed, the reaction itself is converted from an EcoCyc_Reaction to a Reaction object and added to the collection. Throughout this process, errors are logged, particularly when standardization or molecule conversion fails. NOTE: This version requires using the Chemical objects with the HAS_NS_INCHI, HAS_SMILES, and HAS_SMIRKS.


In [10]:
## ------- CONCRETE ONLY -------


# chemicals_by_inchi = {}  # Dictionary: Key = standardized InChI, Value = Chemical object
# reactions_list = []     # List to store Reaction objects

# for ecocyc_reaction in ecocyc_reactions_dict.values():
#     new_substrates = []
#     new_products = []
#     reaction_valid = True

#     # Standardize substrates
#     for substrate_id in ecocyc_reaction.LEFT:
#         ecocyc_chem = ecocyc_chemicals_dict.get(substrate_id)
#         if ecocyc_chem is None or ecocyc_chem.INCHI is None:
#             reaction_valid = False
#             break
#         standardized_inchi, _ = std(ecocyc_chem.INCHI, return_intermediate=True)
#         if standardized_inchi is None:
#             reaction_valid = False
#             break
#         chemical_obj = chemicals_by_inchi.get(standardized_inchi)
#         if chemical_obj is None:
#             # Create a new Chemical object if not found and add it to the dictionary
#             chemical_obj = Chemical(UNIQUE_ID=substrate_id, INCHI=standardized_inchi, SMILES=ecocyc_chem.SMILES, COMMON_NAME=ecocyc_chem.COMMON_NAME)
#             chemicals_by_inchi[standardized_inchi] = chemical_obj

#         new_substrates.append(chemical_obj)  # Add the Chemical object to substrates

#     # Standardize products, similar process as for substrates
#     if reaction_valid:
#         for product_id in ecocyc_reaction.RIGHT:
#             ecocyc_chem = ecocyc_chemicals_dict.get(product_id)
#             if ecocyc_chem is None or ecocyc_chem.INCHI is None:
#               reaction_valid = False
#               break
#             standardized_inchi, _ = std(ecocyc_chem.INCHI, return_intermediate=True)
#             if standardized_inchi is None:
#                 reaction_valid = False
#                 break
#             chemical_obj = chemicals_by_inchi.get(standardized_inchi)
#             if chemical_obj is None:
#                 chemical_obj = Chemical(UNIQUE_ID=product_id, INCHI=standardized_inchi, SMILES=ecocyc_chem.SMILES, COMMON_NAME=ecocyc_chem.COMMON_NAME)
#                 chemicals_by_inchi[standardized_inchi] = chemical_obj

#             new_products.append(chemical_obj)  # Add the Chemical object to products

#     # Create a new Reaction object if all substrates and products are valid
#     if reaction_valid:
#         new_reaction = Reaction(
#             UNIQUE_ID=ecocyc_reaction.UNIQUE_ID,
#             SUBSTRATES=tuple(new_substrates), # Convert list to tuple
#             PRODUCTS=tuple(new_products),     # Convert list to tuple
#             REACTION_DIRECTION=ecocyc_reaction.REACTION_DIRECTION
#         )
#         reactions_list.append(new_reaction)

# print(f"Total RDKit errors encountered: {error_count}")

In [11]:
# ------- RETAINS CLASS COMPOUNDS -------


reactions_list = []  # To store Reaction objects
chemicals_by_inchi = {}  # Dictionary: Key = standardized InChI, Value = Chemical object

for ecocyc_reaction in ecocyc_reactions_dict.values():
    new_substrates = []
    new_products = []
    reaction_valid = True

    # Processing substrates
    for substrate_id in ecocyc_reaction.LEFT:
        ecocyc_chem = ecocyc_chemicals_dict.get(substrate_id)
        if not ecocyc_chem:
            reaction_valid = False
            break

        chemical_obj = None
        key = None
        has_ns_inchi = has_smiles = has_smirks = False

        # Process standard InChI
        if ecocyc_chem.INCHI:
            standardized_inchi, _ = std(ecocyc_chem.INCHI, return_intermediate=True)
            if standardized_inchi:
                key = standardized_inchi
                if key not in chemicals_by_inchi:
                    chemical_obj = Chemical(
                        UNIQUE_ID=substrate_id,
                        COMMON_NAME=ecocyc_chem.COMMON_NAME,
                        SMILES=ecocyc_chem.SMILES,
                        INCHI=standardized_inchi,
                        NON_STANDARD_INCHI='',
                        HAS_NS_INCHI=False,
                        HAS_SMILES=False,
                        HAS_SMIRKS=False
                    )
                    chemicals_by_inchi[key] = chemical_obj
                else:
                    chemical_obj = chemicals_by_inchi[key]

        # Process non-standard InChI if standard InChI is not available
        if not key and ecocyc_chem.NON_STANDARD_INCHI:
            mol = Chem.MolFromInchi(ecocyc_chem.NON_STANDARD_INCHI)
            if mol:
                key = ecocyc_chem.NON_STANDARD_INCHI
                has_ns_inchi = True
                if key not in chemicals_by_inchi:
                    chemical_obj = Chemical(
                        UNIQUE_ID=substrate_id,
                        COMMON_NAME=ecocyc_chem.COMMON_NAME,
                        SMILES='',
                        INCHI='',
                        NON_STANDARD_INCHI=ecocyc_chem.NON_STANDARD_INCHI,
                        HAS_NS_INCHI=True,
                        HAS_SMILES=False,
                        HAS_SMIRKS=False
                    )
                    chemicals_by_inchi[key] = chemical_obj
                else:
                    chemical_obj = chemicals_by_inchi[key]

        # Process SMILES if no InChI is available
        if not key and ecocyc_chem.SMILES:
            mol = Chem.MolFromSmiles(ecocyc_chem.SMILES)
            if mol:
                key = ecocyc_chem.SMILES
                has_smiles = True
            else:
                mol = Chem.MolFromSmarts(ecocyc_chem.SMILES)
                if mol:
                    key = ecocyc_chem.SMILES  # Using SMILES as the key for SMARTS
                    has_smirks = True

            if (has_smiles or has_smirks) and key not in chemicals_by_inchi:
                chemical_obj = Chemical(
                    UNIQUE_ID=substrate_id,
                    COMMON_NAME=ecocyc_chem.COMMON_NAME,
                    SMILES=ecocyc_chem.SMILES,
                    INCHI='',
                    NON_STANDARD_INCHI='',
                    HAS_NS_INCHI=False,
                    HAS_SMILES=has_smiles,
                    HAS_SMIRKS=has_smirks
                )
                chemicals_by_inchi[key] = chemical_obj

        if chemical_obj:
            new_substrates.append(chemical_obj)
        else:
            reaction_valid = False
            break

    # Processing products
    if reaction_valid:  # Only process products if all substrates are valid
        for product_id in ecocyc_reaction.RIGHT:
            ecocyc_chem = ecocyc_chemicals_dict.get(product_id)
            if not ecocyc_chem:
                reaction_valid = False
                break

            chemical_obj = None
            key = None
            has_ns_inchi = has_smiles = has_smirks = False

            # Process standard InChI
            if ecocyc_chem.INCHI:
                standardized_inchi, _ = std(ecocyc_chem.INCHI, return_intermediate=True)
                if standardized_inchi:
                    key = standardized_inchi
                    if key not in chemicals_by_inchi:
                        chemical_obj = Chemical(
                            UNIQUE_ID=product_id,
                            COMMON_NAME=ecocyc_chem.COMMON_NAME,
                            SMILES=ecocyc_chem.SMILES,
                            INCHI=standardized_inchi,
                            NON_STANDARD_INCHI='',
                            HAS_NS_INCHI=False,
                            HAS_SMILES=False,
                            HAS_SMIRKS=False
                        )
                        chemicals_by_inchi[key] = chemical_obj
                    else:
                        chemical_obj = chemicals_by_inchi[key]

            # Process non-standard InChI if standard InChI is not available
            if not key and ecocyc_chem.NON_STANDARD_INCHI:
                mol = Chem.MolFromInchi(ecocyc_chem.NON_STANDARD_INCHI)
                if mol:
                    key = ecocyc_chem.NON_STANDARD_INCHI
                    has_ns_inchi = True
                    if key not in chemicals_by_inchi:
                        chemical_obj = Chemical(
                            UNIQUE_ID=product_id,
                            COMMON_NAME=ecocyc_chem.COMMON_NAME,
                            SMILES='',
                            INCHI='',
                            NON_STANDARD_INCHI=ecocyc_chem.NON_STANDARD_INCHI,
                            HAS_NS_INCHI=True,
                            HAS_SMILES=False,
                            HAS_SMIRKS=False
                        )
                        chemicals_by_inchi[key] = chemical_obj
                    else:
                        chemical_obj = chemicals_by_inchi[key]

            # Process SMILES if no InChI is available
            if not key and ecocyc_chem.SMILES:
                mol = Chem.MolFromSmiles(ecocyc_chem.SMILES)
                if mol:
                    key = ecocyc_chem.SMILES
                    has_smiles = True
                else:
                    mol = Chem.MolFromSmarts(ecocyc_chem.SMILES)
                    if mol:
                        key = ecocyc_chem.SMILES  # Using SMILES as the key for SMARTS
                        has_smirks = True

                if (has_smiles or has_smirks) and key not in chemicals_by_inchi:
                    chemical_obj = Chemical(
                        UNIQUE_ID=product_id,
                        COMMON_NAME=ecocyc_chem.COMMON_NAME,
                        SMILES=ecocyc_chem.SMILES,
                        INCHI='',
                        NON_STANDARD_INCHI='',
                        HAS_NS_INCHI=False,
                        HAS_SMILES=has_smiles,
                        HAS_SMIRKS=has_smirks
                    )
                    chemicals_by_inchi[key] = chemical_obj

            if chemical_obj:
                new_products.append(chemical_obj)
            else:
                reaction_valid = False
                break

    if reaction_valid:
        new_reaction = Reaction(
            UNIQUE_ID=ecocyc_reaction.UNIQUE_ID,
            SUBSTRATES=tuple(new_substrates),
            PRODUCTS=tuple(new_products),
            EC_NUMBER=ecocyc_reaction.EC_NUMBER,
            REACTION_DIRECTION=ecocyc_reaction.REACTION_DIRECTION
        )
        reactions_list.append(new_reaction)

print(f"Total RDKit errors encountered: {error_count}")

Total RDKit errors encountered: 0


####Troubleshooting Tautomerization Errors

This code aims to print out a chemical object in the middle of the standardization (std function) code. For instance, I often struggled placing the standardization order so that the tautomerization worked correctly, so I moved around the intermediate inchi to see what the moleucle looked like at what stage in the process. Although this is labeled for tautomerization, you could use this for other things as well.

In [12]:
def map_inchis_to_ecocyc_info(tautomerization_failures, ecocyc_chemicals_dict):
    failed_chemicals_details = []

    # Iterate through all failed InChI tuples
    for original_inchi, intermediate_inchi in tautomerization_failures:
        # Search for the failed original InChI in the ecocyc_chemicals_dict
        for chemical in ecocyc_chemicals_dict.values():
            if chemical.INCHI == original_inchi:
                # Store the unique ID, common name, original InChI, and transformed InChI for chemicals that match the failed original InChI
                failed_chemicals_details.append((chemical.UNIQUE_ID, chemical.COMMON_NAME, original_inchi, intermediate_inchi))
                break  # Stop searching once a match is found

    return failed_chemicals_details

# Usage example
failed_chemicals_details = map_inchis_to_ecocyc_info(tautomerization_failures, ecocyc_chemicals_dict)

# Print or process the failed chemicals details
for unique_id, common_name, original_inchi, intermediate_inchi in failed_chemicals_details:
    print(f"Failed Chemical: Unique ID = {unique_id}, Common Name = {common_name}, Original InChI = {original_inchi}, Transformed InChI = {intermediate_inchi}")

In [13]:
print(len(tautomerization_failures))
print(len(failed_chemicals_details))

0
0


####Troubleshooting Class Compounds

This section of code is only used only for taking the route that involves the RETAINS CLASS COMPOUNDS filtering strategy. It allowed me to differentiate between what chemicals are referenced in the reactions that are concrete (with an Inchi) and what chemicals are considered "class" compounds and only have a NS-inchi.

In [14]:
print(f"Number of Reaction objects in 'reactions_list': {len(reactions_list)}")
print(f"Number of Reaction objects from the raw data: {len(ecocyc_reactions_dict)}")
print(f"Number of Chemical objects in 'chemicals_by_inchi': {len(chemicals_by_inchi)}")
print(f"Number of Chemical objects from the raw data: {len(ecocyc_chemicals_dict)}")

Number of Reaction objects in 'reactions_list': 2212
Number of Reaction objects from the raw data: 3212
Number of Chemical objects in 'chemicals_by_inchi': 1595
Number of Chemical objects from the raw data: 7585


In [15]:
has_ns_inchi_count = 0
has_smiles_count = 0
has_smirks_count = 0
has_inchi_count = 0

for chem in chemicals_by_inchi.values():
    if chem.HAS_NS_INCHI:
        has_ns_inchi_count += 1
    if chem.HAS_SMILES:
        has_smiles_count += 1
    if chem.HAS_SMIRKS:
        has_smirks_count += 1
    if not chem.HAS_NS_INCHI and not chem.HAS_SMILES and not chem.HAS_SMIRKS:
        has_inchi_count += 1

print(f"Chemicals with non-standard InChI: {has_ns_inchi_count}")
print(f"Chemicals with SMILES: {has_smiles_count}")
print(f"Chemicals with SMIRKS: {has_smirks_count}")
print(f"Chemicals with InChI: {has_inchi_count}")

Chemicals with non-standard InChI: 329
Chemicals with SMILES: 2
Chemicals with SMIRKS: 9
Chemicals with InChI: 1255


In [ ]:
def get_chemicals_by_flag(flag_name):
    result = []
    for key, chem in chemicals_by_inchi.items():
        if getattr(chem, flag_name, False):
            result.append((chem.UNIQUE_ID, chem.COMMON_NAME))
    return result

# Example usage:
ns_inchi_chemicals = get_chemicals_by_flag('HAS_NS_INCHI')
for unique_id, common_name in ns_inchi_chemicals:
    print(f"UNIQUE_ID: {unique_id}, COMMON_NAME: {common_name}")

### Accounting for Reversible and Right-to-Left Reactions

For reversible reactions, this function appends the original reaction to the preprocessed list and also creates a reversed version of it (Unique_ID becomes UNIQUE_ID + "_reverse"), treating the products as substrates and vice versa, marking the direction as left-to-right. For reactions that are inherently right-to-left, the function reverses them to be left-to-right without duplicating them. The preprocessed list then becomes the reactions list based on the function call so that I can be used as a repository for the synthesis algorithm.



In [17]:
def preprocess_reactions(reactions_list):
    preprocessed_reactions = []

    for reaction in reactions_list:
        # Handle reversible reactions by adding a reverse reaction
        if reaction.REACTION_DIRECTION == "REVERSIBLE":
            preprocessed_reactions.append(reaction)  # Original reaction
            # Clone and reverse the original reaction
            reversed_reaction = Reaction(
                UNIQUE_ID=reaction.UNIQUE_ID + "_reverse",  # Ensure unique ID for the reversed reaction
                SUBSTRATES=reaction.PRODUCTS,
                PRODUCTS=reaction.SUBSTRATES,
                EC_NUMBER=reaction.EC_NUMBER,
                REACTION_DIRECTION="LEFT-TO-RIGHT"  # Assuming reversed is always left-to-right
            )
            preprocessed_reactions.append(reversed_reaction)

        # Handle right-to-left reactions by reversing them
        elif reaction.REACTION_DIRECTION == "PHYSIOL-RIGHT-TO-LEFT" or reaction.REACTION_DIRECTION == "RIGHT-TO-LEFT":
            reversed_reaction = Reaction(
                UNIQUE_ID=reaction.UNIQUE_ID,
                SUBSTRATES=reaction.PRODUCTS,
                PRODUCTS=reaction.SUBSTRATES,
                EC_NUMBER=reaction.EC_NUMBER,
                REACTION_DIRECTION="LEFT-TO-RIGHT"
            )
            preprocessed_reactions.append(reversed_reaction)
        else:
            # For left-to-right and other directions, add the reaction as is
            preprocessed_reactions.append(reaction)

    return preprocessed_reactions

reactions_list = preprocess_reactions(reactions_list)

# Incorporating the Universal and Minimal Metabolites

### Loading in Minimal and Universal Metabolites

In [32]:
def load_metabolites(file_path):
    metabolites_inchis = []
    with open(file_path, 'r') as file:
        lines = file.read().splitlines()  # Read the whole file as a string and split into lines
    for i in range(1, len(lines)):  # Start from 1 to skip the header
        parts = lines[i].split('\t') # Split each line at tab characters, resulting in a list of values
        inchi = parts[1]
        inchi = inchi.replace('"','') # Remove any double quotes from the InChI string for standardization
        metabolites_inchis.append(inchi) # Add the cleaned InChI string to the list of metabolite InChIs

    return metabolites_inchis

minimal_inchis = load_metabolites('minimal_metabolites_02-24.txt') #Call the function. List is now titled minimal_inchis
universal_inchis = load_metabolites('universal_metabolites_03-24.txt')
native_inchis = minimal_inchis + universal_inchis #creating a master list

#Test master list
print(native_inchis[:5])
len(native_inchis)


['InChI=1S/C6H12O6/c7-1-2-3(8)4(9)5(10)6(11)12-2/h2-11H,1H2/t2-,3-,4+,5-,6?/m1/s1', 'InChI=1S/H3N/h1H3/p+1', 'InChI=1S/Na/q+1', 'InChI=1S/K/q+1', 'InChI=1S/Ca/q+2']


88

### Standardizing and Curating Native Chemicals List

This code aims to identify the cooresponding Chemical object within Chemicals_by_inchi by doing 1-to-1 InChI matching. It then logs those Chemical objects into a new list called updated_native_chemicals. These are all directly logged into Shell 0 in the synthesis algorithm. The InChIs that don't have any cooresponding chemical object are logged into the unresolved_metabolites list.

In [33]:
updated_native_chemicals = []
unresolved_metabolites = []

for native_inchi in native_inchis:
    standardized_inchi = std(native_inchi)  # Standardize each native InChI
    if standardized_inchi in chemicals_by_inchi:
        updated_native_chemicals.append(chemicals_by_inchi[standardized_inchi]) #updated_native_chemicals is now a list of chemical objects for native chemicals
    else:
        unresolved_metabolites.append(native_inchi)  # Log unresolved metabolites

print("InChI strings found after standardization:", updated_native_chemicals, "\n", len(updated_native_chemicals))
print("InChI strings still unresolved after standardization:", unresolved_metabolites, "\n", len(unresolved_metabolites))
for chem_obj in updated_native_chemicals:
  print(f"{chem_obj.UNIQUE_ID}: {chem_obj.COMMON_NAME}")


InChI strings found after standardization: [Chemical(UNIQUE_ID='Glucopyranose', COMMON_NAME='D-glucopyranose', SMILES='C([C@@H]1([C@H]([C@@H]([C@H](C(O1)O)O)O)O))O', INCHI='InChI=1S/C6H14O6/c7-1-3(9)5(11)6(12)4(10)2-8/h3-12H,1-2H2/t3-,5-,6-/m1/s1', NON_STANDARD_INCHI='', HAS_NS_INCHI=False, HAS_SMILES=False, HAS_SMIRKS=False), Chemical(UNIQUE_ID='AMMONIUM', COMMON_NAME='ammonium', SMILES='[NH4+]', INCHI='InChI=1S/H3N/h1H3', NON_STANDARD_INCHI='', HAS_NS_INCHI=False, HAS_SMILES=False, HAS_SMIRKS=False), Chemical(UNIQUE_ID='NA+', COMMON_NAME='Na<SUP>+</SUP>', SMILES='[Na+]', INCHI='InChI=1S/Na/q+1', NON_STANDARD_INCHI='', HAS_NS_INCHI=False, HAS_SMILES=False, HAS_SMIRKS=False), Chemical(UNIQUE_ID='K+', COMMON_NAME='K<SUP>+</SUP>', SMILES='[K+]', INCHI='InChI=1S/K/q+1', NON_STANDARD_INCHI='', HAS_NS_INCHI=False, HAS_SMILES=False, HAS_SMIRKS=False), Chemical(UNIQUE_ID='CA+2', COMMON_NAME='Ca<SUP>2+</SUP>', SMILES='[Ca+2]', INCHI='InChI=1S/Ca/q+2', NON_STANDARD_INCHI='', HAS_NS_INCHI=False,

###Troubleshooting Unresolved Metabolites

This code just aims to figure out the metadata of the unresolved metabolites.

In [34]:
def load_metabolites_dict(file_path):
    metabolites_named_inchis = {}
    with open(file_path, 'r') as file:
        lines = file.read().splitlines()  # Read the whole file as a string and split into lines
    for i in range(1, len(lines)):  # Start from 1 to skip the header
        parts = lines[i].split('\t') # Split each line at tab characters, resulting in a list of values
        name = parts[0]
        inchi = parts[1]
        inchi = inchi.replace('"','') # Remove any double quotes from the InChI string for standardization
        metabolites_named_inchis[name]=inchi # Add name as key and the cleaned InChI string as value to the named metabolite InChIs dictionary

    return metabolites_named_inchis

minimal_named_inchis = load_metabolites_dict('minimal_metabolites_02-24.txt') #Call the function. Dictionary is now titled minimal_inchis
universal_named_inchis = load_metabolites_dict('universal_metabolites_03-24.txt')
native_named_inchis = {**minimal_named_inchis, **universal_named_inchis} #Merges both dictionaries with values from universal_named_inchis overriding those from minimal_named_inchis if there are duplicate keys.

not_found_metabolite_names = []  # List to store keys whose values match the InChI strings in not_found_inchis
for inchi in unresolved_metabolites:
    # Iterate over each key-value pair in the native_named_inchis dictionary
    for key, value in native_named_inchis.items():
        if inchi == value:
            not_found_metabolite_names.append(key)

not_found_metabolite_dict = {}  # List to store keys whose values match the InChI strings in not_found_inchis
for inchi in unresolved_metabolites:
    # Iterate over each key-value pair in the native_named_inchis dictionary
    for key, value in native_named_inchis.items():
        if inchi == value:
            not_found_metabolite_dict[key] = value

print("Names of Unmatched Metabolites:", not_found_metabolite_names)
print(("Names of Unmatched Metabolites with InChI:", not_found_metabolite_dict))

Names of Unmatched Metabolites: ['molybdenum (+1)', 'molybdenum (+2)', 'molybdenum (+6)', 'nickel (+1)', 'iodide', 'thiamine pyrophosphate (p+1)', 'NAD+ (p+1)', 'FAD', 'FAD (3)', 'tetrahydrofolic acid (t11+)', 'ubiquinone-9', 'ubiquinol', 'heme', 'tetrahydrobiopterin', 'GGPP']
('Names of Unmatched Metabolites with InChI:', {'molybdenum (+1)': 'InChI=1S/Mo/q+1', 'molybdenum (+2)': 'InChI=1S/Mo/q+2', 'molybdenum (+6)': 'InChI=1S/Mo/q+6', 'nickel (+1)': 'InChI=1S/Ni/q+1', 'iodide': 'InChI=1S/HI/h1H/p-1', 'thiamine pyrophosphate (p+1)': 'InChI=1S/C12H18N4O7P2S/c1-8-11(3-4-22-25(20,21)23-24(17,18)19)26-7-16(8)6-10-5-14-9(2)15-12(10)13/h5,7H,3-4,6H2,1-2H3,(H4-,13,14,15,17,18,19,20,21)/p+1', 'NAD+ (p+1)': 'InChI=1S/C21H27N7O14P2/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(32)14(30)11(41-21)6-39-44(36,37)42-43(34,35)38-5-10-13(29)15(31)20(40-10)27-3-1-2-9(4-27)18(23)33/h1-4,7-8,10-11,13-16,20-21,29-32H,5-6H2,(H5-,22,23,24,25,33,34,35,36,37)/p+1/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1', 'FAD': 'InC

# Synthesis

Comparing this to Steury Synthesis:

**Initialization**:

Steury: Had an empty initiate method, suggesting it was intended for future implementation or customization.

Mine: Includes a defined __init__ method initializing the list for reactions and dictionary for the chemicals, reflecting a more structured approach to managing synthesis data.



In [39]:
# HyperGraph class to store relationships and states of chemicals and reactions
class HyperGraph:
    def __init__(self):
        self.reaction_to_shell = {}  # Dictionary mapping each reaction to its respective shell (depth level in the synthesis)
        self.chemical_to_shell = {}  # Dictionary mapping each chemical to its respective shell
        self.chemical_to_cascade = {}  # Additional data structure, if needed
        self.chemical_to_pathway = {}  # Additional data structure, if needed

# Synthesizer class handling the synthesis algorithm
class Synthesizer:
    def __init__(self):
        self.curr_shell = 0
        self.all_reactions = []  # This will be populated with Reaction objects
        self.all_chemicals = {}  # Dictionary: Key = standardized InChI, Value = Chemical object
        self.chemical_to_shell = {}  # Now tracking Chemical objects directly
        self.reaction_to_shell = {}

    # Main method to run the synthesis process
    def run(self, reactions_list, chemicals_by_inchi, updated_native_chemicals):
        self.curr_shell = 0
        self.all_reactions = reactions_list
        self.all_chemicals = chemicals_by_inchi

        #log native chemicals into shell 0
        for native_chem in updated_native_chemicals:
          self.chemical_to_shell[native_chem] = 0

        # Expand the synthesis process shell by shell
        while self._expand_once():
            print(self.curr_shell)

        # Compile results in a HyperGraph object
        output = HyperGraph()
        output.chemical_to_shell = self.chemical_to_shell
        output.reaction_to_shell = self.reaction_to_shell
        return output

    def _expand_once(self):
        self.curr_shell += 1
        is_expanded = False

        # Temporary storage for reactions enabled in this iteration
        enabled_reactions = []

        # Iterate through all reactions
        for reaction in self.all_reactions:
            # Skip if the reaction has already been put in the expansion
            if reaction in self.reaction_to_shell:
                continue

            # Check if all substrates are available in previous shells
            substrates_available = all(
                self.chemical_to_shell.get(substrate) is not None and
                self.chemical_to_shell[substrate] < self.curr_shell  # Ensure substrates were available before current shell
                for substrate in reaction.SUBSTRATES
            )

            if substrates_available:
                is_expanded = True
                # Temporarily store the reaction as enabled
                enabled_reactions.append(reaction)

        # After checking all reactions, update the shells for enabled reactions and their products
        for reaction in enabled_reactions:
            self.reaction_to_shell[reaction] = self.curr_shell
            for product in reaction.PRODUCTS:
                if product not in self.chemical_to_shell:
                    self.chemical_to_shell[product] = self.curr_shell

        return is_expanded

# Usage example
synthesizer = Synthesizer()
hypergraph = synthesizer.run(reactions_list, chemicals_by_inchi, updated_native_chemicals)

#print(hypergraph.chemical_to_shell)
#print(hypergraph.reaction_to_shell)

size = len(hypergraph.chemical_to_shell)
print(f"Total Size of Chemical to Shell: {size}")
rxn_size = len(hypergraph.reaction_to_shell)
print(f"Total Size of Reaction to Shell: {rxn_size}")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Total Size of Chemical to Shell: 600
Total Size of Reaction to Shell: 1450


# Queries for Hypergraph Analysis

###Reaction ID in Hypergraph

In [40]:
# Checking if reactions made it into my hypergraph
unique_id_to_check = 'GLYOHMETRANS-RXN'

# Initialize a variable to store the found reaction details
found_reaction_details = None

# Iterate through the reaction_to_shell dictionary
for reaction, shell_number in hypergraph.reaction_to_shell.items():
    if reaction.UNIQUE_ID == unique_id_to_check:
        # If the reaction is found, store its details including shell number, reactants, and products
        found_reaction_details = {
            "shell_number": shell_number,
            "reactants": [reactant.UNIQUE_ID for reactant in reaction.SUBSTRATES],
            "products": [product.UNIQUE_ID for product in reaction.PRODUCTS],
            "direction": reaction.REACTION_DIRECTION
        }
        break  # Stop the loop once the reaction is found

# Check if the reaction was found and print details
if found_reaction_details:
    print(f"Reaction {unique_id_to_check} is in the HyperGraph in shell {found_reaction_details['shell_number']}.")
    print(f"Reactants: {', '.join(found_reaction_details['reactants'])}")
    print(f"Products: {', '.join(found_reaction_details['products'])}")
    print(f"Direction: {found_reaction_details['direction']}")
else:
    print(f"Reaction {unique_id_to_check} is not in the HyperGraph.")

Reaction GLYOHMETRANS-RXN is not in the HyperGraph.


###List of Reactions per Shell

In [41]:
def list_reactions_by_shell(hypergraph, shells_to_list):
    """
    Lists reactions within specified shells along with their substrates and products.

    Parameters:
    - hypergraph: An instance of the HyperGraph class.
    - shells_to_list: A list of integers representing the shell numbers to list reactions for.
    """
    for shell_number in shells_to_list:
        print(f"Reactions in Shell {shell_number}:")
        for reaction, shell in hypergraph.reaction_to_shell.items():
            if shell == shell_number:
                substrates = ', '.join([substrate.UNIQUE_ID for substrate in reaction.SUBSTRATES])
                products = ', '.join([product.UNIQUE_ID for product in reaction.PRODUCTS])
                print(f"- {reaction.UNIQUE_ID}: Substrates: [{substrates}] -> Products: [{products}]")
        print("\n")  # Add a newline for better readability between shells

# Example usage:
list_reactions_by_shell(hypergraph, [20])

Reactions in Shell 20:
- RXN-14188: Substrates: [5-HYDROXY-CTP, WATER] -> Products: [CPD-15158, PPI, PROTON]
- RXN-11396: Substrates: [8-Oxo-dGTP, WATER] -> Products: [PROTON, CPD-12365, PPI]
- RXN0-6957: Substrates: [CPD-13851, WATER] -> Products: [CPD-13852, PPI, PROTON]
- RXN-11397: Substrates: [CPD-12366, WATER] -> Products: [PROTON, CPD-12367, PPI]
- ADOMET-DMK-METHYLTRANSFER-RXN: Substrates: [S-ADENOSYLMETHIONINE, CPD-12115] -> Products: [REDUCED-MENAQUINONE, PROTON, ADENOSYL-HOMO-CYS]
- RXN0-6708: Substrates: [PROTON, CPD0-2461, WATER] -> Products: [XANTHINE, AMMONIUM]




###Chemical ID in Hypergraph

In [42]:
def find_chemical_by_id(hypergraph, unique_id_to_check):
    """
    Finds a chemical by its unique ID and prints the shell it is in.

    Parameters:
    - hypergraph: An instance of the HyperGraph class.
    - unique_id_to_check: The unique ID of the chemical to find.
    """
    for chemical, shell_number in hypergraph.chemical_to_shell.items():
        if chemical.UNIQUE_ID == unique_id_to_check:
            print(f"Chemical {unique_id_to_check} is in Shell {shell_number}.")
            return
    print(f"Chemical {unique_id_to_check} is not in the HyperGraph.")

# Example usage:
find_chemical_by_id(hypergraph, "SER")

Chemical SER is not in the HyperGraph.


###List of Chemicals per Shell

In [43]:
def list_chemicals_by_shells(hypergraph, shells_to_list):
    """
    Lists all chemicals within specified shells.

    Parameters:
    - hypergraph: An instance of the HyperGraph class.
    - shells_to_list: A list of integers representing the shell numbers to list chemicals for.
    """
    for shell_number in shells_to_list:
        print(f"Chemicals in Shell {shell_number}:")
        for chemical, shell in hypergraph.chemical_to_shell.items():
            if shell == shell_number:
                print(f"- {chemical.UNIQUE_ID} (Shell {shell})")
        print("\n")  # Add a newline for better readability between shells

# Example usage:
list_chemicals_by_shells(hypergraph, [18])

Chemicals in Shell 18:
- CPD-12377 (Shell 18)
- DIHYDROXYNAPHTHOATE (Shell 18)
- OCTAPRENYL-METHYL-METHOXY-BENZQ (Shell 18)
- SUPER-OXIDE (Shell 18)


